**It's time to EXPLORE**

# Exploration 3: Heart Failure

<font color="Green">KaggleStreak </font>

So, here I continue my **Exploration Series**:
 *" It consists of Data Pre-processing, Data Visualization, EDA, Feature Engg., Model build and finally Evaluation of model. "*

**This is 2nd in series**

1)[🔦Exploration : Placement Data](https://www.kaggle.com/iabhishekmaurya/exploration-placement-data)

2)[🔦Exploration :  Microsoft Store](https://www.kaggle.com/iabhishekmaurya/exploration-2-microsoft-store)

Coming to the present dataset I a using **[Heart Failure](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data)** dataset

This dataset was developed by *Larxel*


So, here I start my exploration to the saame dataset.

Do give <font color="Red"> UPVOTE </font>


# Libraries/Modules Imports

In [ ]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from imblearn.over_sampling import RandomOverSampler
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, classification_report, accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC

from xgboost import XGBClassifier

# Data Preprocessing

In [ ]:
# Reading the dataset
heart_data = pd.read_csv('../input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')
heart_data.head()

In [ ]:
print("Shape of data: ",heart_data.shape)

In [ ]:
heart_data.info()

**So clearly we don't have any null value in any of the column.**

In [ ]:
heart_data['time'].describe()

In [ ]:
heart_data['DEATH_EVENT'].value_counts()

**We have around 2:1 ratio for targeted column, so we would be requiring to work on data imbalance**(worked later)

# Data Visualization + EDA

**Ploting data gives us a brief understanding of data, so let's use it**

First, we need to find list of column name we have

In [ ]:
heart_data.columns

## Age

In [ ]:
var = 'age'
fig, ax = plt.subplots()
fig.set_size_inches(20, 8)
plt.xticks(rotation=90);
sns.countplot(x = var,palette="ch:.4", data = heart_data)
ax.set_xlabel('AGE', fontsize=15)
ax.set_ylabel('Count', fontsize=15)
ax.set_title('AGE Count Distribution', fontsize=15)
sns.despine()

## Death

In [ ]:
var = 'DEATH_EVENT'
fig, ax = plt.subplots()
fig.set_size_inches(11.7, 8.27)
sns.swarmplot(x = var, y ='age', data = heart_data)

## Ejection Fraction

In [ ]:
heart_data['ejection_fraction'].describe()

In [ ]:
var = 'age'
data = pd.concat([heart_data['ejection_fraction'], heart_data[var]], axis=1)
f, ax = plt.subplots(figsize=(20, 10))
fig = sns.boxplot(x=var, y="ejection_fraction", data=data)
fig.axis(ymin=0, ymax=90);
plt.xticks(rotation=90);

## Anaemia

In [ ]:
var = 'anaemia'
fig, ax = plt.subplots()
fig.set_size_inches(5,5)
sns.countplot(x = var, data = heart_data)
ax.set_xlabel('anaemia', fontsize=15)
ax.set_ylabel('Count', fontsize=15)
ax.set_title('anaemia Count Distribution', fontsize=15)
ax.tick_params(labelsize=15)
sns.despine()

## *var* count basis of diabetes

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(5,5)
sns.countplot(x='DEATH_EVENT',hue='diabetes', data=heart_data)
ax.set_xlabel('DEATH_EVENT', fontsize=15)
ax.set_ylabel('Count', fontsize=15)
ax.set_title('DEATH_EVENT count basis of diabetes', fontsize=10)
ax.tick_params(labelsize=15)
plt.show() 

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20, 8)
sns.countplot(x='ejection_fraction',hue='diabetes', data=heart_data)
ax.set_xlabel('ejection_fraction', fontsize=15)
ax.set_ylabel('Count', fontsize=15)
ax.set_title('ejection_fraction count basis of diabetes', fontsize=10)
ax.tick_params(labelsize=15)
plt.show() 

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows = 2, ncols = 2, figsize = (10,10))

sns.countplot(x='DEATH_EVENT',hue='diabetes', data=heart_data, ax = ax1)
ax1.set_xlabel('DEATH_EVENT')
ax1.set_ylabel('Count')
ax1.set_title('DEATH_EVENT count basis of diabetes', fontsize=10)
ax1.tick_params(labelsize=15)

sns.countplot(x='high_blood_pressure',hue='diabetes', data=heart_data, ax = ax2)
ax2.set_xlabel('high_blood_pressure')
ax2.set_ylabel('Count')
ax2.set_title('DEATH_EVENT count basis of diabetes', fontsize=10)
ax2.tick_params(labelsize=15)

sns.countplot(x='sex',hue='diabetes', data=heart_data, ax = ax3)
ax3.set_xlabel('sex')
ax3.set_ylabel('Count')
ax3.set_title('sex count basis of diabetes', fontsize=10)
ax3.tick_params(labelsize=15)

sns.countplot(x='smoking',hue='diabetes', data=heart_data, ax = ax4)
ax4.set_xlabel('smoking')
ax4.set_ylabel('Count')
ax4.set_title('smoking count basis of diabetes', fontsize=10)
ax4.tick_params(labelsize=15)

plt.subplots_adjust(wspace=0.5)
plt.tight_layout() 

## Platelets

In [ ]:
heart_data['platelets'].describe()

In [ ]:
f, ax = plt.subplots(figsize=(15,8))
sns.distplot(heart_data['platelets'])
plt.xlim([0,900000])

## Heat Map

In [ ]:
plt.figure(figsize=(18,18))
sns.heatmap(heart_data.corr(),annot=True,cmap='RdYlGn')

plt.show()

# Train Test Split

In [ ]:
X = heart_data.drop(['DEATH_EVENT'], axis = 1)
y = heart_data['DEATH_EVENT']

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X , y, test_size=0.2, random_state=25)

# Improving data imbalance

In [ ]:
os=RandomOverSampler(1)
X_train_ns,y_train_ns=os.fit_sample(X_train,y_train)
print("The number of classes before fit {}".format(Counter(y_train)))
print("The number of classes after fit {}".format(Counter(y_train_ns)))

# Model Evaluation

In [ ]:
models = []
models.append(('LR', LogisticRegression(solver='liblinear')))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('DTC', DecisionTreeClassifier()))
models.append(('SVM', SVC(gamma='auto')))
models.append(('RFC', RandomForestClassifier()))
models.append(('ABC', AdaBoostClassifier()))
models.append(('XGB', XGBClassifier()))
# evaluate each model in turn
results = []
names = []
for name, model in models:
    kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
    cv_results = cross_val_score(model, X_train_ns, y_train_ns, cv=kfold, scoring='accuracy')
    results.append(cv_results)
    names.append(name)
    print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

## <font style="color:#9e7fec">That's it for today, you could do more work upon it like:</font>
* **Working over Model Selection**: Finding out which model work best.
* **Working with Model Parameters**: After selecting the best model one could find respective best parameters using RandomSearchCV, GridSearchCv, and others.

**Quotes:**

**“The purpose of our lives is to be happy.” — *Dalai Lama***

<font style="color:orange;font-size:18px"><b>A Humble Request: IF you liked my work </b></font>
<font style="color:red;font-size:18px"><b>DO GIVE UPVOTE</b></font>

<font style="color:green;font-size:18px">And Guess What, It won't cost any, instead Motivates ME</font>

### Till then:
 <font style="color:#7fecde; font-size:20px;line-height:24px;">My other Work:<ul>
  <li><a href='https://www.kaggle.com/iabhishekmaurya/used-car-price-prediction'>Used Car Price Pridiction </a><a href='https://used-car-price-pred.herokuapp.com/'>  (Deployed Link)</a></li>
  <li><a href='https://www.kaggle.com/iabhishekmaurya/exploration-placement-data'>Exploration Series Part 1</a></li>
  <li><a href='https://www.kaggle.com/iabhishekmaurya/case-study-cholera'>Case Study Series Part 1</a></li>
  <li><a href='https://www.kaggle.com/iabhishekmaurya/notebooks'>More NoteBooks</a></li>
</ul> </font>
<hr>

<font style="color:green;font-size:18px">BE SAFE & HAPPY CODING 👨🏻‍💻</font>